In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load datasets
mentors = pd.read_csv("Mentor.csv")  # Columns: Name, Sector 1, Sector 2, Sector 3, Index
startups = pd.read_csv("Startups.csv")  # Columns: Name, Sector, Index

# Initialize time slots
start_time = datetime.strptime("11:00 AM", "%I:%M %p")
end_time = datetime.strptime("2:00 PM", "%I:%M %p")
time_slots = []
slot_duration = timedelta(minutes=15)
gap_duration = timedelta(minutes=5)

while start_time + slot_duration <= end_time:
    time_slots.append(start_time.strftime("%I:%M %p"))
    start_time += slot_duration + gap_duration

# Helper function to assign startups to mentors based on preferences
def assign_startups(mentors, startups):
    mentor_schedule = {mentor["Name"]: [] for _, mentor in mentors.iterrows()}
    startup_counts = {startup["Name"]: 0 for _, startup in startups.iterrows()}
    assigned_startups = set()
    
    for _, mentor in mentors.iterrows():
        preferences = [mentor["Sector 1"], mentor["Sector 2"], mentor["Sector 3"]]
        weights = [60, 24, 16]
        mentor_startups = []

        # Assign startups based on sector preference
        for _ in range(len(time_slots)):
            available_startups = startups[
                ~startups["Name"].isin([s["Name"] for s in mentor_startups]) &
                startups["Sector"].isin(preferences)
            ]

            if available_startups.empty:
                continue

            # Select a startup based on weighted preferences
            chosen_sector = random.choices(preferences, weights=weights, k=1)[0]
            sector_startups = available_startups[available_startups["Sector"] == chosen_sector]
            
            if sector_startups.empty:
                continue
            
            startup = sector_startups.sample(1).iloc[0]
            mentor_startups.append({"Name": startup["Name"], "Sector": startup["Sector"]})
            assigned_startups.add(startup["Name"])
            startup_counts[startup["Name"]] += 1

            # Stop assigning if startup count exceeds limit
            if startup_counts[startup["Name"]] >= 4:
                break

        mentor_schedule[mentor["Name"]] = mentor_startups

    # Reallocate startups if necessary
    for _, startup in startups.iterrows():
        if startup["Name"] not in assigned_startups:
            for mentor in mentors["Name"]:
                if startup_counts[startup["Name"]] < 4:
                    mentor_schedule[mentor].append({"Name": startup["Name"], "Sector": startup["Sector"]})
                    assigned_startups.add(startup["Name"])
                    startup_counts[startup["Name"]] += 1
                    break

    return mentor_schedule, startup_counts

# Assign startups to mentors
schedule, startup_counts = assign_startups(mentors, startups)

# Assign time slots to mentors
final_schedule = []
for mentor, startups in schedule.items():
    slots = iter(time_slots)
    for startup in startups:
        final_schedule.append({
            "Mentor": mentor,
            "Startup": startup["Name"],
            "Sector": startup["Sector"],
            "Time Slot": next(slots, "N/A")
        })

# Display the schedule
print("Mentoring Schedule:")
for _, mentor in mentors.iterrows():
    print(f"\nMentor: {mentor['Name']}")
    print(f"Preferences: {mentor['Sector 1']}, {mentor['Sector 2']}, {mentor['Sector 3']}")
    print("Scheduled Startups:")
    for session in [s for s in final_schedule if s["Mentor"] == mentor["Name"]]:
        print(f"  - {session['Startup']} ({session['Sector']}) at {session['Time Slot']}")

# Display startup counts
print("\nStartup Session Counts:")
for startup, count in startup_counts.items():
    print(f"{startup}: {count} sessions")


Mentoring Schedule:

Mentor: Mentor 1
Preferences: B2B Software, Marketplace, Media/Media Tech
Scheduled Startups:
  - Startup 128 (B2B Software) at 11:00 AM
  - Startup 76 (Media/Media Tech) at 11:20 AM
  - Startup 38 (Marketplace) at 11:40 AM
  - Startup 69 (Media/Media Tech) at 12:00 PM
  - Startup 141 (Marketplace) at 12:20 PM
  - Startup 88 (Marketplace) at 12:40 PM
  - Startup 96 (Marketplace) at 01:00 PM
  - Startup 41 (B2B Software) at 01:20 PM
  - Startup 15 (Marketplace) at 01:40 PM
  - Startup 2 (Agriculture) at N/A
  - Startup 3 (Blockchain) at N/A
  - Startup 9 (Fintech) at N/A
  - Startup 12 (Agriculture) at N/A
  - Startup 23 (Marketplace) at N/A
  - Startup 29 (Agriculture) at N/A
  - Startup 31 (Marketplace) at N/A
  - Startup 39 (HR Tec) at N/A
  - Startup 50 (Agriculture) at N/A
  - Startup 51 (Energy And Environment) at N/A
  - Startup 52 (Automotive) at N/A
  - Startup 53 (Agriculture) at N/A
  - Startup 68 (Marketplace) at N/A
  - Startup 94 (Agriculture) at N/A
 